In [ ]:
# imports
import requests
import json

import open3d as o3d
import numpy as np

import subprocess


In [ ]:

# get the id of the first annotation
url = "http://localhost:4321/api/queries/findAnnotation"
response = requests.get(url)
id = json.loads(response.text)[0]
print(id)

In [10]:
tgt_points_id = "3d87d707-5735-4d7d-a401-679ae2a748b5"
src_points_id = "4b854809-1601-4e7e-aa78-85aa19fa43d3"

In [13]:
# Define the URL and query parameters
url = "http://localhost:4322/api/annotations/3d87d707-5735-4d7d-a401-679ae2a748b5/points"

# Send a GET request to the Suave endpoint
response = requests.get(url)
tgt_points = np.array(json.loads(response.text))

print(f"tgt points: {tgt_points}")

# Define the URL and query parameters
url = "http://localhost:4321/api/annotations/4b854809-1601-4e7e-aa78-85aa19fa43d3/points"

# Send a GET request to the Suave endpoint
response = requests.get(url)
src_points = np.array(json.loads(response.text))

print(f"src points: {src_points}")


tgt points: [[137.15062074 -56.11408989 -29.09428223]
 [108.25235213 -46.44734016 -27.29873521]
 [ 72.28858124 -58.32665047 -23.85722113]
 [  4.47920139 -91.81704426 -42.33971145]]
src points: [[ 701757.85653848 3142014.52990013 1073527.84141308]
 [ 701746.07743941 3142024.38444733 1073498.90227216]
 [ 701763.76885274 3142030.26883148 1073463.93951211]
 [ 701813.23370933 3142061.36943788 1073394.22420414]]


In [14]:
# Convert numpy arrays to Open3D PointClouds
source_pcd = o3d.geometry.PointCloud()
source_pcd.points = o3d.utility.Vector3dVector(src_points)

target_pcd = o3d.geometry.PointCloud()
target_pcd.points = o3d.utility.Vector3dVector(tgt_points)

# Define correspondences as index pairs
correspondences = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
correspondences_o3d = o3d.utility.Vector2iVector(correspondences)

# Create a TransformationEstimationPointToPoint object
estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

# Compute the rigid transformation matrix
transformation_matrix = estimation.compute_transformation(target_pcd, source_pcd, correspondences_o3d)

# Print the transformation matrix
print("Computed Transformation Matrix:")
print(transformation_matrix)

Computed Transformation Matrix:
[[-9.55238971e-02 -9.57203477e-01 -2.73197160e-01  7.01709083e+05]
 [-3.07990171e-01  2.89405582e-01 -9.06303737e-01  3.14204795e+06]
 [ 9.46581872e-01 -2.43162464e-03 -3.22454412e-01  1.07338495e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [15]:
# Apply transformation to target surface
tgt_surface_id = "191cef90-bcbc-4acb-9318-ae11aa6355d4"

url = f"http://localhost:4321/api/surfaces/{tgt_surface_id}/transform"

payload = {
    "forward": transformation_matrix.tolist()
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
response.raise_for_status() 
print(response.text)

Received payload [[-0.09552389707154708, -0.9572034771126825, -0.2731971604750966, 701709.0825239161], [-0.30799017131390394, 0.2894055815028879, -0.9063037370379803, 3142047.9521466875], [0.9465818715052057, -0.002431624635177196, -0.32245441187761237, 1073384.950562644], [0, 0, 0, 1]]


In [16]:
cut_annotation_id = "4c2224dd-96d8-4921-824b-f82f2c1e3812"
url = "http://localhost:4321/api/queries/queryAnnotationAsObj"

# retrieve the cutout geometry
payload = {
    "annotationId": cut_annotation_id, 
    "queryAttributes":[], 
    "distanceToPlane": 100000.0,
    "outputReferenceFrame": "global",
    "outputGeometryType": "mesh"
}
print(payload)
response = requests.post(url, json=payload)
response.raise_for_status() 
filename = "./coutout.obj"
file = open(filename, "w")
file.write(response.text)
file.close()

{'annotationId': '4c2224dd-96d8-4921-824b-f82f2c1e3812', 'queryAttributes': [], 'distanceToPlane': 100000.0, 'outputReferenceFrame': 'global', 'outputGeometryType': 'mesh'}


In [25]:
tgt_filePath = "G:\PRo3D\AI-Mars-3D\Co-Registration\packageGP\m2020-zcam-kodiak\source\model_4b_kodiak_100m_zcam_scam.obj"
tgt_filePath_trans = "G:\PRo3D\AI-Mars-3D\Co-Registration\packageGP\m2020-zcam-kodiak\source\model_4b_kodiak_100m_zcam_scam_IAU.obj"

mesh = o3d.io.read_triangle_mesh(tgt_filePath)

mesh.vertices = o3d.utility.Vector3dVector(np.asarray(mesh.vertices, dtype=np.float64))

mesh = mesh.transform(transformation_matrix)
o3d.io.write_triangle_mesh(tgt_filePath_trans, mesh)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D WARNING] Write PNG failed: image has no data.
[Open3D WARNING] Write OBJ successful, but failed to write texture file.


True

In [34]:
# ICP fine registration

# Load the SRC and TGT meshes
src_mesh = o3d.io.read_triangle_mesh(filename)
tgt_mesh = o3d.io.read_triangle_mesh("G:\PRo3D\AI-Mars-3D\Co-Registration\packageGP\m2020-zcam-kodiak\source\model_4b_kodiak_100m_zcam_scam.obj")

# Convert meshes to point clouds for registration
src_pcd = src_mesh.sample_points_uniformly(number_of_points=10000)
tgt_pcd = tgt_mesh.sample_points_uniformly(number_of_points=10000)


# Compute normals for both point clouds
src_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
tgt_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Define the pre-alignment (Helmert transformation)
pre_alignment = transformation_matrix

# Apply the pre-alignment transformation to the TGT point cloud
tgt_pcd.transform(pre_alignment)

output_file = "./output_pointcloud.obj"
with open(output_file, "w") as f:
    for point in tgt_pcd.points:
        f.write(f"v {point[0]} {point[1]} {point[2]}\n")
        
# Perform fine registration using ICP
icp_result = o3d.pipelines.registration.registration_icp(
    tgt_pcd, src_pcd, max_correspondence_distance=0.1,
    init=np.eye(4),
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
)

# Print the transformation matrices
print("Pre-Alignment (Helmert) Transformation:")
print(pre_alignment)

print("\nICP Refinement Transformation:")
print(icp_result.transformation)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
Pre-Alignment (Helmert) Transformation:
[[-9.55238971e-02 -9.57203477e-01 -2.73197160e-01  7.01709083e+05]
 [-3.07990171e-01  2.89405582e-01 -9.06303737e-01  3.14204795e+06]
 [ 9.46581872e-01 -2.43162464e-03 -3.22454412e-01  1.07338495e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

ICP Refinement Transformation:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  2.84786044e-10  0.00000000e+00]
 [ 0.00000000e+00 -2.84786044e-10  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [35]:
# open cutout in MeshLab
try:
    subprocess.Popen(["C:/Program Files/vcg/MeshLab/meshlab.exe", output_file])
    print("MeshLab started successfully.")
except FileNotFoundError:
    print("MeshLab executable not found. Please ensure it is installed and in your system PATH.")

MeshLab started successfully.


In [ ]:
# Apply final trafo to target surface
tgt_surface_id = "191cef90-bcbc-4acb-9318-ae11aa6355d4"

url = f"http://localhost:4321/api/surfaces/{tgt_surface_id}/transform"

payload = {
    "forward": icp_result.transformation.tolist()
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
response.raise_for_status() 
print(response.text)

In [ ]:
# GREETING TEST

# Define the URL and query parameters
url = "http://localhost:4322/api/exp/greet/Orti"
url = "http://localhost:4322/api/exp/greet"
params = {"name": "Orti"}

# Send a GET request to the Suave endpoint
response = requests.get(url, params=params)

# Check the response
if response.status_code == 200:
    print(f"Response: {response.text}")
else:
    print(f"Error: {response.status_code}, Message: {response.text}")